In [1]:
# %load masterOOScript.py
"""
Created on Tue Jul 19 16:14:43 2016

@author: nmvenuti
"""

# -*- coding: utf-8 -*-
"""
Created on Thu Jun  2 15:23:11 2016

@author: nmvenuti
"""
import time
start=time.time()
import sys, os

os.chdir('/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017') ##### CHANGE TO YOUR WORKING DIRECTORY
#from joblib import Parallel, delayed
#import multiprocessing as mp
import os.path
import numpy as np
import pandas as pd
from datetime import datetime
sys.path.append('./prototype_python/')
import lingualPrinting as la
import nltk
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('averaged_perceptron_tagger')


end=time.time()
#sys.stdout = open("output.txt", "a")
print(str(datetime.now()))
print('finished loading packages after '+str(end-start)+' seconds')
sys.stdout.flush()


stemmer = nltk.stem.snowball.EnglishStemmer()

#for generating id's for different test runs
import random
import string
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

[nltk_data] Downloading package punkt to /Users/Seth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/Seth/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Seth/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
2016-10-15 16:01:31.271552
finished loading packages after 2.0028719902 seconds


In [2]:
print(type(la.posWords))
print(la.posWords)
print(la.negWords) ## WHY IS EACH LETTER PAIR REVERSED?

<type 'list'>
[u'\n', u'+', u'-', u'a', u'c', u'b', u'e', u'd', u'g', u'f', u'i', u'h', u'k', u'j', u'm', u'l', u'o', u'n', u'q', u'p', u's', u'r', u'u', u't', u'w', u'v', u'y', u'x', u'z']
[u'\n', u'*', u'-', u'2', u'a', u'c', u'b', u'e', u'd', u'g', u'f', u'i', u'h', u'k', u'j', u'm', u'l', u'o', u'n', u'q', u'p', u's', u'r', u'u', u't', u'w', u'v', u'y', u'x', u'z']


In [15]:
##########################
#####Define Functions#####
##########################
def textAnalysis(paramList):
    startTime=time.time()
    groupId=paramList[0]
    fileList=paramList[1]
    targetWordCount=paramList[2]
    cocoWindow=paramList[3]
    svdInt=paramList[4]
    cvWindow=paramList[5]
    simCount=paramList[6]
    startCount=paramList[7]
    netAngle=paramList[8]    
    
    #Get list of subfiles
    subFileList=[x[1] for x in fileList if x[0]==groupId[0] and x[2]==groupId[1]]
    #print(subFileList)
    
    #Create lingual object
    loTest=la.lingualObject(subFileList)

    #Get coco
    loTest.getCoco(cocoWindow)
    
    #Get DSM
    loTest.getDSM(svdInt)
    
    #Set keywords
    loTest.setKeywords('adjAdv',targetWordCount,startCount)

    #######################            
    ###Semantic analysis###
    #######################
    
    #Get context vectors
    loTest.getContextVectors(cvWindow)
    
    #Get average semantic density
    avgSD=np.mean([x[1] for x in loTest.getSD(simCount)])
    
    ########################################
    ###POS Tagging and Judgement Analysis###
    ########################################
    judgementAvg=list(np.mean(np.array([[x[1],x[2]] for x in loTest.getJudgements()]),axis=0))
    
    ########################
    ###Sentiment Analysis###
    ########################

    sentimentList=loTest.sentimentLookup()
    
    ############################
    ###Network Quantification###
    ############################
    loTest.setNetwork(netAngle)
    
    avgEVC=loTest.evc()
    
    endTime=time.time()
    timeRun=endTime-startTime
    print('finished running'+'_'.join(groupId)+' in '+str(end-start)+' seconds')
    sys.stdout.flush()

    #Append outputs to masterOutput
    print(['_'.join(groupId)]+[len(subFileList),timeRun]+sentimentList+judgementAvg+[avgSD]+[avgEVC])   
    return(['_'.join(groupId)]+[len(subFileList),timeRun]+sentimentList+judgementAvg+[avgSD]+[avgEVC])   

def runMaster(rawPath,groupList,crossValidate,groupSize,targetWordCount,startCount,cocoWindow,svdInt,cvWindow,netAngle,simCount):
    ###############################
    #####Raw File List Extract#####
    ###############################

    rawFileList=[]
    for groupId in groupList:
        for dirpath, dirnames, filenames in os.walk(rawPath+groupId+'/raw'):
            for filename in [f for f in filenames ]:
                if '.txt' in filename:
                    rawFileList.append([groupId,os.path.join(dirpath, filename)])

    #Make output directory
    runDirectory='./pythonOutput/TEST-coco_'+str(cocoWindow)+'_cv_'+str(cvWindow)+'_netAng_'+str(netAngle)+'_sc_'+str(startCount)+id_generator()
    os.makedirs(runDirectory)
    print(runDirectory)
    end=time.time()
    print('finished loading packages after '+str(end-start)+' seconds')
    sys.stdout.flush()
    
    
    #Perform analysis for each fold in cross validation
    for fold in range(crossValidate):                
        ###############################                
        #####Set up random binning#####
        ###############################
                        
        fileDF=pd.read_csv('./data_dsicap/test_train/fileSplit_'+str(fold)+'.csv')
        print(fileDF.head())
        
        fileList=fileDF.values.tolist()
        #print(fileList) #########
        fileList=[[fileList[i][1],fileList[i][2],fileList[i][3]] for i in range(len(fileList))]
        #print(fileList) ############
        
        #Get set of subgroups
        subgroupList=[ list(y) for y in set((x[0],x[2]) for x in fileList) ]
        print(subgroupList)
        
        #Make output directory
        outputDirectory=runDirectory+'/run'+str(fold)
        os.makedirs(outputDirectory)
        
        #Print file splits to runDirectory
        fileDF.to_csv(outputDirectory+'/fileSplits.csv')

        end=time.time()
        print('finished randomly creating subgroups '+str(end-start)+' seconds')
        sys.stdout.flush()        
        
        ################################
        #####Perform group analysis#####
        ################################
        
        #Create paramList
        paramList=[[x,fileList,targetWordCount,cocoWindow,svdInt,cvWindow,simCount,startCount,netAngle] for x in subgroupList]
        print(paramList)
        
        #Run calculation 
        masterOutput=[textAnalysis(x) for x in paramList]  
        #print(masterOutput)
        
        #Create output file
        outputDF=pd.DataFrame(masterOutput,columns=['groupId','files','timeRun','perPos','perNeg','perPosDoc','perNegDoc','judgementCount','judgementFrac','avgSD','avgEVC'])
        outputDF.to_csv(outputDirectory+'/masterOutput.csv')
    print("%%%%%%%%%%%%%%\nAll done.\n$$$$$$$$$$$$$$$$")

#Set inital conditions and run
if __name__ == '__main__':
    startTimeTotal=time.time()
    rawPath = './data_dsicap/'
#    groupList=['DorothyDay','JohnPiper','MehrBaba','NaumanKhan','PastorAnderson',
#       'Rabbinic','Shepherd','Unitarian','WBC']
    groupList=['NaumanKhan', 'Rabbinic', 'NawDawg']
    
#    cocoWindow=int(sys.argv[1])
#    cvWindow=int(sys.argv[2])
#    startCount=int(sys.argv[3])
#    netAngle=int(sys.argv[4])
    cocoWindow=3
    cvWindow=3
    startCount=0
    netAngle=30
    crossValidate=1 # IF YOU CHANGE THIS, YOU HAVE TO PUT YOUR FILES IN THE data_dsicap/test_train/fileSplit_#.csv FILES
    groupSize=10
#    testSplit=.3
    targetWordCount=10
    svdInt=50
    simCount=1000
    print('cocoWindow '+str(cocoWindow))
    sys.stdout.flush()
    print('cvWindow '+str(cvWindow))
    sys.stdout.flush()
    print('netAngle '+str(netAngle))
    sys.stdout.flush()
    print('startCount '+str(startCount))
    sys.stdout.flush()
    
    runMaster(rawPath,groupList,crossValidate,groupSize,targetWordCount,startCount,cocoWindow,svdInt,cvWindow,netAngle,simCount)
        
    endTimeTotal=time.time()
    print('finished entire run in :'+str((endTimeTotal-startTimeTotal)/60)+' minutes')
    sys.stdout.flush()

cocoWindow 3
cvWindow 3
netAngle 30
startCount 0
./pythonOutput/TEST-coco_3_cv_3_netAng_30_sc_0MLQQQY
finished loading packages after 2353.68671799 seconds
      X       group                                           filepath  \
0  1618  NaumanKhan  ./data_dsicap/NaumanKhan/raw/from_darkness_to_...   
1  1619  NaumanKhan  ./data_dsicap/NaumanKhan/raw/brilliance_of_the...   
2  1620  NaumanKhan      ./data_dsicap/NaumanKhan/raw/khutbah.docx.txt   
3  1621  NaumanKhan  ./data_dsicap/NaumanKhan/raw/lessons_from_a_bi...   
4  1622  NaumanKhan  ./data_dsicap/NaumanKhan/raw/shield-of-honor-r...   

  subgroup  
0  train04  
1  train04  
2  train01  
3  train00  
4  train04  
[['Rabbinic', 'test04'], ['Rabbinic', 'train06'], ['NaumanKhan', 'train00'], ['Rabbinic', 'train09'], ['NaumanKhan', 'train01'], ['Rabbinic', 'train00'], ['Rabbinic', 'train11'], ['NaumanKhan', 'train03'], ['NaumanKhan', 'train04'], ['NawDawg', 'train100'], ['Rabbinic', 'train08']]
finished randomly creating subgroups 2